In [ ]:
import pandas as pd
import pickle
import re
import openai
from bs4 import BeautifulSoup
from tqdm import tqdm

# Preprocessing

In [ ]:
def summarize(prompt, query, model):
    query = re.sub('\\s{2,}', '', query)
    query = f'''
    {prompt}
    
    {query}
    '''

    messages = [{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": query
    }]

    return openai.ChatCompletion.create(model=model, messages=messages)

In [ ]:
if __name__ == '__main__':
    data_path = ''
    data = pd.read_pickle(data_path).get('q1')
    data = {key: value for key, value in data.items() if value.get('company_info')}
    
    OPENAI_API_KEY = ''
    openai.api_key = OPENAI_API_KEY
    model = "gpt-3.5-turbo-0125"
    
    for i, (key, value) in enumerate(tqdm(data.items())):
        try:
            company_info_summarized = summarize(
                '회사 주요 내용 짧게 요약해줘',
                BeautifulSoup(value.get('company_info'), 'lxml').text,
                model,
                )
        except:
            company_info_summarized = ''
            
        try:
            business_info_summarized = summarize(
                '사업 주요 내용 짧게 요약해줘',
                BeautifulSoup(value.get('business_info'), 'lxml').text,
                model,
                )
        except:
            business_info_summarized = ''
        
        value.update({
            'company_info_summarized': company_info_summarized,
            'business_info_summarized': business_info_summarized,
        })
        
        if i%5 == 0:
            with open('./reports_1_summarize_by_gpt.pkl', 'wb') as file:
                pickle.dump(data, file)

# Check

In [28]:
data = pd.read_pickle('./reports_1_summarize_by_gpt.pkl')
data_w_summarization = {key: value for key, value in data.items() if value.get('company_info_summarized')}
data_wo_summarization = {key: value for key, value in data.items() if not value.get('company_info_summarized')}

In [3]:
data_wo_summarization.keys()
# 휴비스, 휴스틸, 흥국화재, 흥아해운

dict_keys(['079980', '005010', '000540', '003280'])

In [27]:
def extract_information(item: dict, target: str):
    text = item.get(target)
    if item.get(target):
        text = text.get('choices')[0].get('message').get('content')

    return text

In [29]:
for key, value in tqdm(data_w_summarization.items()):
    value.update({
        'company_info_summarized': extract_information(value, 'company_info_summarized'),
        'business_info_summarized': extract_information(value, 'business_info_summarized'),
    })

100%|██████████| 786/786 [00:00<00:00, 105011.24it/s]


In [33]:
data_w_summarization.get('034310').get('company_info_summarized')

'회사는 주식회사 나이스홀딩스로 알려져 있으며, 주요 사업으로는 지주회사로써 자회사의 경영을 지원하는 홀딩스사업을 주로 하고 있습니다. 현재 39개의 종속회사를 보유하고 있으며, 주요 사업으로는 기업 및 개인신용정보사업, 신용카드 거래승인사업, CD/ATM 관리사업, 신용평가사업, 자산관리사업, 제조업 등을 영위하고 있습니다. 그 외에도 부동산 임대, 채권평가 및 컨설팅, 데이터분석, 경영컨설팅 등 다양한 사업을 진행하고 있습니다.회사는 2004년 유가증권시장에 상장되었으며, 현재 국내에서 각종 금융 및 비금융사업을 다각화하여 운영하고 있습니다. 또한, 마이데이터 사업 진출로 신규 사업 영역을 확대하고 있으며, 꾸준한 성장이 기대됩니다.회사 주소는 서울 영등포구에 위치하고 있으며, 홈페이지 주소는 http://www.nice.co.kr 입니다.'

In [35]:
data_w_summarization.get('034310').get('business_info_summarized')

주요 내용을 정리하면 아래와 같습니다:
- 회사는 자회사의 지배와 경영을 위한 지주회사업을 주요 사업으로 하며, 다양한 분야의 종속회사를 보유하고 있음.
- 종속회사들은 기업정보, 개인신용정보 사업, CD/ATM 관리, 금융서비스 제공, 무인주차장 운영, KIOSK 사업, 현금물류 등 다양한 사업을 영위.
- 주요 종속회사로는 NICE평가정보㈜, 한국전자금융㈜, NICE정보통신㈜, 나이스인프라㈜, 케이아이에스정보통신㈜,㈜제이티넷, NICE페이먼츠㈜, ITM Semiconductor VIETNAM Co., Ltd., (주)NICE엘엠에스, LMS VINA Co., Ltd. 등이 있음.
- 주요 사업으로는 정보제공, 금융서비스 제공, 부동산임대, 전자금융, 알루미늄 합금 제품 제조 등이 있음.


In [36]:
with open('./reports_1_summarize_by_gpt_and_clean.pkl', 'wb') as file:
    pickle.dump(data_w_summarization, file)